# Code bits used in the documentation for Andante

## Parser example from section 1.3

In [1]:
from andante.program import AndanteProgram 
apmovies = AndanteProgram.build_from("Examples/movies.pl")
H = apmovies.induce(update_knowledge=True, logging=True, verbose=0)

In [2]:
H

Knowledge object (class: TreeShapedKnowledge)
Clauses:
   recommend(A, B) :- b45to49(A).
   recommend(A, B) :- b18to24(A), drama(B).
   recommend(A, B) :- crime(B).
   recommend(A, B) :- horror(B).
   recommend(A, B) :- b25to34(A), f(A).
   recommend(A, B) :- adventure(B).

In [33]:
text = """
:- begin_bg.
    user(u777).
    b18to24(u777).
    f(u777).
    movie(m777).
    drama(m777).
    action(m1374).
action(m1676).
adventure(m1259).
adventure(m2987).
b18to24(u1433).
b18to24(u1778).
b18to24(u2777).
b18to24(u3418).
b18to24(u4103).
b18to24(u4411).
b18to24(u621).
b25to34(u2840).
b25to34(u3123).
b25to34(u3539).
b25to34(u4138).
b25to34(u4560).
b25to34(u4607).
b25to34(u5077).
b35to44(u1197).
b45to49(u5127).
b50to55(u4981).
comedy(m19).
comedy(m2596).
comedy(m2888).
comedy(m3909).
comedy(m691).
crime(m47).
drama(m1178).
drama(m1619).
drama(m17).
drama(m1960).
drama(m337).
drama(m924).
f(u1433).
f(u3418).
f(u3539).
horror(m1348).
m(u1197).
m(u1778).
m(u2777).
m(u2840).
m(u3123).
m(u4083).
m(u4103).
m(u4138).
m(u4411).
m(u4560).
m(u4607).
m(u4981).
m(u5077).
m(u5127).
m(u621).
plus56(u4083).
sci_fi(m1206).
    recommend(A, B) :- b45to49(A).
    recommend(A, B) :- b18to24(A), drama(B).
    recommend(A, B) :- crime(B).
    recommend(A, B) :- horror(B).
    recommend(A, B) :- b25to34(A), f(A).
    recommend(A, B) :- adventure(B).
:- end_bg.
"""
ap2 = AndanteProgram.build_from(text)
ap2.query('recommend(u777, B).')

(True,
       0
 B  m777)

In [17]:
from andante.parser import Parser
parser = Parser()
userbk = """
:- begin_bg.
    b18to25(u777).
    f(u777).
    movie(m777).
    drama(m777).
    recommend(X, Y) :- b18to24(X), drama(Y).
    recommend(X, Y) :- b25to34(X), f(X).
:- end_bg.
 """
bknowledge = parser.parse(userbk, rule='background')
print(bknowledge)

Knowledge object (class: TreeShapedKnowledge)
Clauses:
   b18to25(u777).
   f(u777).
   movie(m777).
   drama(m777).
   recommend(X, Y) :- b18to24(X), drama(Y).
   recommend(X, Y) :- b25to34(X), f(X).


In [18]:
from andante.solver import AndanteSolver
solver = AndanteSolver()
query = parser.parse("recommend(A, B).", rule="query")

# Using method succeeds_on
success = solver.succeeds_on(query, bknowledge)
if success:
    print("The query succeeded")
else:
    print("The query failed")
    
# Using method query
print("The solutions for the query are:")
for solution in solver.query(query, bknowledge):
    print('\t', solution)

The query failed
The solutions for the query are:


In [1]:
from andante.parser import Parser
parser = Parser()
text = """
:- begin_bg.
    mortal(X):-man(X).
    man(socrates).
:- end_bg.
"""
knowledge = parser.parse(text, rule='background')
print(knowledge)

Knowledge object (class: TreeShapedKnowledge)
Clauses:
   mortal(X) :- man(X).
   man(socrates).


## Solver example from section 2.2

In [2]:
from andante.solver import AndanteSolver
solver = AndanteSolver()
q = parser.parse("mortal(X).", rule="query")

# Using method succeeds_on
success = solver.succeeds_on(q, knowledge)
if success:
    print("The query succeeded")
else:
    print("The query failed")
    
# Using method query
print("The solutions for the query are:")
for solution in solver.query(q, knowledge):
    print('\t', solution)

The query succeeded
The solutions for the query are:
	 {X: socrates}


## Learner example from section 3.3

In [3]:
text = """
% Mode declarations
modeh(*,daughter(+person,-person)).
modeb(*,parent(+person,-person)).
modeb(*,parent(-person,+person)).
modeb(*,female(+person)).
modeb(*,female(-person)).

% Determinations
determination(daughter/2,parent/2).
determination(daughter/2,female/1).

% Background knowledge
:- begin_bg.
person(ann). person(mary). person(tom). person(eve). person(lucy).
female(ann). female(mary). female(eve). female(lucy).
parent(ann,mary).
parent(ann,tom).
parent(tom,eve).
parent(tom,lucy).
:- end_bg.
    
% Positive examples
:- begin_in_pos.
daughter(lucy,tom).
daughter(mary,ann).
daughter(eve,tom).
:- end_in_pos.

% Negative examples
:- begin_in_neg.
daughter(tom,ann).
daughter(tom,eve).
:- end_in_neg.
"""
pr = parser.parse(text, rule="andantefile")
H = pr.induce()

print("Knowledge learned")
print(H)

Knowledge learned
Knowledge object (class: TreeShapedKnowledge)
Clauses:
   daughter(A, B) :- female(A).


## Interface example from section 4

In [3]:
from andante.program import AndanteProgram
from andante.interface import MainInterface

ap = AndanteProgram.build_from("Examples/movies.pl")
interface = MainInterface(ap)
interface.widget